## extra_reprメソッドとその周辺

extra_reprメソッドは，nn.Moduleサブクラスのインスタンスをprintした際に表示される文字列を設定するメソッドです．\__str\__や\__repr\__のように一般的なPythonクラスに用意された特殊メソッドではなく，nn.Module独自のメソッドになります．

- `extra_repr`メソッド
    - クラスの名前や内部のパラメータの形状などを出力させます．
- `__str__`メソッド
    - 人間にとってわかりやすく，画面に表示しやすいオブジェクトの表現を出力させます．（した方がいいです）
- `__repr__`メソッド
    - eval関数で評価すると元のオブジェクトに戻るような，Pythonが元のオブジェクトを復元できるオブジェクト表現を出力させます．（した方がいいです）

### 自作クラスとオブジェクトid

何の設定もしていない自作クラスがどのように表示されるかを見てみましょう：  

In [3]:
class temp:
    ...

obj = temp()

print("普通にprintした時に表示されるもの:\t",obj)
print("オブジェクトのid:\t\t\t",id(obj))
print("オブジェクトのidを16進数にしたもの:\t", hex(id(obj)))

普通にprintした時に表示されるもの:	 <__main__.temp object at 0x104e7b370>
オブジェクトのid:			 4377260912
オブジェクトのidを16進数にしたもの:	 0x104e7b370


特に何の設定もしていない自作クラスをprintすると，オブジェクトのid（識別子）が16進数で表示されるようです．また，おそらく皆さんはCPythonを利用しているはずなので，このオブジェクトidにはメモリアドレスが利用されています．（id関数のdocstringを読んでみてください）

In [5]:
id?

Signature: id(obj, /)
Docstring:
Return the identity of an object.

This is guaranteed to be unique among simultaneously existing objects.
(CPython uses the object's memory address.)
Type:      builtin_function_or_method

> これは同時に存在するオブジェクトの中で一意であることが保証されています。(CPythonはオブジェクトのメモリアドレスを使います)。

逆説的に，CPython以外のPython実装では，オブジェクトidがメモリアドレスだとは限らないことに注意です．

### extra_reprの設定

[nn.Linearクラスの実装](https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html#Linear)を参考にextra_reprの書き方を覚えてください．

```python
class Linear(Module):
    r"""Applies a linear transformation to the incoming data: :math:`y = xA^T + b`

    This module supports :ref:`TensorFloat32<tf32_on_ampere>`.

    On certain ROCm devices, when using float16 inputs this module will use :ref:`different precision<fp16_on_mi200>` for backward.

    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        bias: If set to ``False``, the layer will not learn an additive bias.
            Default: ``True``

    Shape:
        - Input: :math:`(*, H_{in})` where :math:`*` means any number of
          dimensions including none and :math:`H_{in} = \text{in\_features}`.
        - Output: :math:`(*, H_{out})` where all but the last dimension
          are the same shape as the input and :math:`H_{out} = \text{out\_features}`.

    Attributes:
        weight: the learnable weights of the module of shape
            :math:`(\text{out\_features}, \text{in\_features})`. The values are
            initialized from :math:`\mathcal{U}(-\sqrt{k}, \sqrt{k})`, where
            :math:`k = \frac{1}{\text{in\_features}}`
        bias:   the learnable bias of the module of shape :math:`(\text{out\_features})`.
                If :attr:`bias` is ``True``, the values are initialized from
                :math:`\mathcal{U}(-\sqrt{k}, \sqrt{k})` where
                :math:`k = \frac{1}{\text{in\_features}}`

    Examples::

        >>> m = nn.Linear(20, 30)
        >>> input = torch.randn(128, 20)
        >>> output = m(input)
        >>> print(output.size())
        torch.Size([128, 30])
    """
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        # Setting a=sqrt(5) in kaiming_uniform is the same as initializing with
        # uniform(-1/sqrt(in_features), 1/sqrt(in_features)). For details, see
        # https://github.com/pytorch/pytorch/issues/57109
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight, self.bias)

    def extra_repr(self) -> str:
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )
```